In [2]:
import heapq
from collections import defaultdict, deque
from typing import Dict, List, Set, Tuple

**332. Reconstruct Itinerary**


Hierholzer's algorithm is a classical algorithm used to find an Eulerian circuit or Eulerian path in a graph. An Eulerian circuit is a closed path that visits every edge in a graph exactly once, starting and ending at the same vertex. An Eulerian path also visits every edge exactly once, but it does not need to start and end at the same vertex.

Prerequisites

Before applying Hierholzer's algorithm, ensure:

For an Eulerian circuit:

-   The graph is connected.
-   Every vertex has an even degree.

For an Eulerian path:

-   The graph is connected.
-   Exactly two vertices have an odd degree (start and end points of the path).

Steps of Hierholzer's Algorithm

-   Start at any vertex with a non-zero degree (for Eulerian circuits) or at a vertex with an odd degree (for Eulerian paths).
-   Follow edges to create a cycle:
    -   While traversing, mark edges as "used" so they aren't revisited.
    -   Stop when you return to the starting vertex (if constructing a circuit) or can't continue further (for paths).
-   Check for unused edges:
    -   If there are unused edges, pick a vertex in the current cycle that connects to an unused edge and repeat the process, forming a new sub-cycle.
-   Merge cycles:
    -   Combine the sub-cycles with the main cycle by splicing them together at the appropriate vertices.
-   Repeat until all edges are used, ensuring the resulting path covers all edges exactly once.


In [ ]:
class Solution:
    def findItinerary(
        self, tickets: List[List[str]]
    ) -> List[str]:  # 80% time, 18% memory
        """
        Read the comments in the recursive version (right below) for a better explanation
        """
        graph = defaultdict(list)
        # reverse the sorted list to allow easier traversal with pop() (as it removes from the end of the list):
        # pop will remove the smallest lexicographical destination since it is reversed!!
        # then that gets processed before the higher lexico destinations.
        for orig, dest in sorted(tickets)[::-1]:
            graph[orig].append(dest)

        # eulerian path:
        res = []
        # iterative version
        stack = ["JFK"]
        while stack:
            curr = stack[-1]
            if not graph[curr]:
                # all children, and all children of children, etc etc, have been processed and added to the result.
                res.append(stack.pop())
            else:
                stack.append(graph[curr].pop())

        return res[::-1]

    def findItineraryRecursion(
        self, tickets: List[List[str]]
    ) -> List[str]:  # 74% time, 10% memory
        graph = defaultdict(list)
        for orig, dest in sorted(tickets)[::-1]:
            graph[orig].append(dest)

        # eulerian circuit:
        res = []

        def dfs(airport):
            # """
            # The DFS explores the graph by traversing all outgoing edges from src. Here's the key insight:
            # - Edges are removed as they're visited using pop(), ensuring no edge is traversed more than once.
            # - Once all edges from a node are visited, that node is added to the res list.

            # This is post order traversal ^
            # """
            while graph[airport]:
                dfs(graph[airport].pop())
            res.append(airport)

        # """
        # DFS appends nodes to res only when all their edges have been processed. This gives the correct Eulerian path but in reverse order (because it's constructed during the backtracking phase of DFS).
        # Reversing res at the end produces the final itinerary.
        # """
        dfs("JFK")
        return res[::-1]

    def findItineraryInitial(self, tickets: List[List[str]]) -> List[str]:
        """
        Passes everythin except the last test case
        """
        # sort to meet the lexicographical constraint
        tickets.sort(key=lambda ticket: str(ticket))

        graph: Dict[str, List[str]] = defaultdict(list)
        for orig, dest in tickets:
            graph[orig].append(dest)

        res = []
        cur = ["JFK"]
        # success base case needs to get this many flights:
        # + 1 because there are (number of tickets + 1) airports in an itinerary
        num_airports_needed = len(tickets) + 1

        def dfs():
            if len(cur) == num_airports_needed:
                res.append(cur)
                return True

            airport = cur[-1]
            dests = graph[airport].copy()
            for i in range(len(dests)):
                graph[airport].pop(i)
                cur.append(dests[i])

                if dfs() is not None:
                    # early exit if success
                    # first success will be the solution thanks to the sort at the beginning
                    return True

                graph[airport].insert(i, dests[i])
                cur.pop()

        dfs()
        return res[0]


findItinerary = Solution()
print(
    findItinerary.findItinerary(
        [["MUC", "LHR"], ["JFK", "MUC"], ["SFO", "SJC"], ["LHR", "SFO"]]
    )
)
print(
    findItinerary.findItinerary(
        [["JFK", "SFO"], ["JFK", "ATL"], ["SFO", "ATL"], ["ATL", "JFK"], ["ATL", "SFO"]]
    )
)

['JFK', 'MUC', 'LHR', 'SFO', 'SJC']
['JFK', 'ATL', 'JFK', 'SFO', 'ATL', 'SFO']
['JFK', 'SFO', 'JFK', 'ATL', 'AAA', 'BBB', 'ATL', 'AAA', 'BBB', 'ATL', 'AAA', 'BBB', 'ATL', 'AAA', 'BBB', 'ATL', 'AAA', 'BBB', 'ATL', 'AAA', 'BBB', 'ATL', 'AAA', 'BBB', 'ATL', 'AAA', 'BBB', 'ATL', 'AAA', 'BBB', 'ATL', 'AAA', 'BBB', 'ATL', 'AAA', 'BBB', 'ATL', 'AAA', 'BBB', 'ATL', 'AAA', 'BBB', 'ATL', 'AAA', 'BBB', 'ATL', 'AAA', 'BBB', 'ATL', 'AAA', 'BBB', 'ATL', 'AAA', 'BBB', 'ATL', 'AAA', 'BBB', 'ATL', 'AAA', 'BBB', 'ATL', 'AAA', 'BBB', 'ATL', 'AAA', 'BBB', 'ATL', 'AAA', 'BBB', 'ATL', 'AAA', 'BBB', 'ATL', 'AAA', 'BBB', 'ATL', 'AAA', 'BBB', 'ATL', 'AAA', 'BBB', 'ATL']


# Minimum Spanning Trees

**Anything related to minimums and graphs/trees is gonna be Prim's**

> Brute-force using a minHeap

Prim's Algorithm `O(v^2 log(v))` v is num of edges

uses a minheap as a Frontier:

In **Prim's algorithm** for finding a **minimum spanning tree (MST)**, the **frontier** refers to the set of edges that connect vertices already included in the MST to vertices that are not yet included.

At each step of Prim's algorithm:

1. You start with an initial vertex (or any random vertex).
2. From this vertex, you examine all the edges that connect to other vertices, forming the initial frontier.
3. The algorithm selects the edge with the smallest weight from the frontier, adds the connected vertex to the MST, and updates the frontier to include new edges from the added vertex.
4. This process repeats until all vertices are included in the MST.

Thus, the **frontier** is the dynamically updated collection of edges that represents potential candidates for inclusion in the MST at any given time.


<br><br>
**1584. Min cost to connect all points**


In [ ]:
class Solution:  # 86% time, 74% memory; Additional attempt after a couple of months (Dec 8, 2024)
    """
    Using Prim's algorithm to get the Minimum Spanning Tree

    Frontier: set of edges that connect vertices already in the MST to vertices not in the MST
        We use a minHeap as the Frontier in Prim's algo so that the smallest-weight-edge is the one that is used every time a new edge is added to the MST

    Can iterate through the points and add the manhattan costs of every possible connection.
        Build graph as you traverse. This works because at each vertex we are finding the smallest connection between it and the rest of the graph.
            This is necessarily going to be an edge in the MST so we can just add it to our result storing the overall weight.

        ^ Need to loop through the other points and calculate the cost of a connection with them

    Can build graph then traverse (I don't like this)

    In either case:
    - Time:
        - Every vertex gets visited `v` times
        - The heap operations are log(v)
        - So overall, v^2 heap operations = O(v^2 * log(v)) time
    - Memory:
        - O(v^2) memory for the frontier
    """

    def minCostConnectPoints(self, points: List[List[int]]) -> int:
        if len(points) == 1:
            return 0

        # know when to stop adding edges to MST
        # will store the index of the node
        processed_vertices: Set[int] = set()
        # the minHeap Frontier
        frontier = []
        # init the return value
        res = 0
        # the current node being processed
        left = 0

        while len(processed_vertices) != len(points):
            # compute all the weights for connections from points[left]
            x, y = points[left]
            processed_vertices.add(left)

            for right in range(len(points)):
                # don't need to evaluate a potential connection with vertex that has already been added to the MST
                # we don't apply this logic when assigning cur_node because the MST can extend beyond cur_node lol.
                if right not in processed_vertices:
                    heapq.heappush(
                        frontier,
                        (
                            abs(x - points[right][0]) + abs(y - points[right][1]),
                            right,
                        ),
                    )
                # else: we have already added the right vertex to MST so no point in checking.

            while frontier[0][1] in processed_vertices:
                # this is important because we are cleaning useless edges, from earlier iterations, to a node that has already been added to MST
                heapq.heappop(frontier)
            weight, right = heapq.heappop(frontier)

            res += weight
            processed_vertices.add(right)

            # move the current node to a new node (can do left += 1)
            left = right

        return res


minCostConnectPoints = Solution()
minCostConnectPoints.minCostConnectPoints([[0, 0], [2, 2], [3, 10], [5, 2], [7, 0]])

20

In [ ]:
class Solution:  # 88% time, 74% memory
    def minCostConnectPoints(self, points: List[List[int]]) -> int:
        # my solution generates the graph as it generates the result.
        if len(points) == 1:
            return 0
        visited = set()
        frontier = []
        heapq.heapify(frontier)

        res = 0
        left = 0

        while len(visited) < len(points):
            cur_node = points[left]
            visited.add(left)
            for right in range(len(points)):
                if right in visited:
                    continue
                node = points[right]
                heapq.heappush(
                    frontier,
                    (abs(node[0] - cur_node[0]) + abs(node[1] - cur_node[1]), right),
                )

            cost, right = heapq.heappop(frontier)
            while right in visited:
                # this is important because we are popping visited nodes added from earlier iterations.
                cost, right = heapq.heappop(frontier)

            # IMPORTANT!
            # the first unvisited popped node will be the shortest path from any of the already visited nodes to the unvisited one
            # this is why we are pushing every possible edge to the heap, the optimal solution might be one central node going out to every other one for example
            # IMPORTANT!
            res += cost
            visited.add(right)
            # technically doing left + 1 will have the same exact result and performance.
            # I just do this because it guarantees to reduce your search space by at least one node
            left = right

        return res

    def minCostConnectPointsNeetCode(self, points: List[List[int]]) -> int:
        N = len(points)
        # generate graph with weights first
        adj = {i: [] for i in range(N)}  # i: list of [cost, node]
        for i in range(N):
            x1, y1 = points[i]
            for j in range(i + 1, N):
                x2, y2 = points[j]
                dist = abs(x1 - x2) + abs(y1 - y2)
                adj[i].append([dist, j])
                adj[j].append([dist, i])

        # Prim's algorithm on generated graph
        res = 0
        visit = set()
        minH = [[0, 0]]
        while len(visit) < N:
            cost, i = heapq.heappop(minH)
            if i in visit:
                continue
            res += cost
            visit.add(i)
            for neiCost, nei in adj[i]:
                if nei not in visit:
                    heapq.heappush(minH, [neiCost, nei])
        return res


S = Solution()
S.minCostConnectPoints([[0, 0], [2, 2], [3, 10], [5, 2], [7, 0]])

**743. Network Delay Time**


In [ ]:
class Solution:
    def networkDelayTime(
        self, times: List[List[int]], n: int, k: int
    ) -> int:  # 65 % time, 22% memory
        """
        Djikstra's BFS with minHeap O(ElogV) time O(V+E) space

        This time I have to build an adjacency graph though with weights
        I will do a BFS with a visited set

        I can definitely make the average time go faster by checking if the graph is disconnected at the beginning.
        Just check if every node 1 to n is another node's destination while you are building the graph

        That is exactly what Claude 3.5 Sonnet suggested as well lol:
        ```
        # Add this check while building the graph
        reachable = set()
        for src, dst, _ in times:
            reachable.add(dst)
        if len(reachable) < n-1:  # Can't reach all nodes
            return -1
        ```
        "This early check could save time on disconnected graphs, though it may slightly slow down the average case."
        """
        # n nodes labelled 1 to n
        # increasing size by one so that i don't have to subtract one for indexing
        # the index of `graph` is the value of the node for that index
        graph: List[List[Tuple[int, int]]] = [[] for _ in range(n + 1)]
        for src, dst, cost in times:
            # append cost, destination (formatted so the frontier can use the cost for heapage)
            graph[src].append((cost, dst))

        visited = set()
        minPath = [(0, k)]  # start at node k
        heapq.heapify(minPath)

        while minPath:
            cur_cost, src = heapq.heappop(minPath)
            visited.add(src)
            # success case
            if len(visited) == n:
                return cur_cost

            for cost, dst in graph[src]:
                if dst not in visited:
                    # no need to push to heap if the destination has already been added to the path
                    heapq.heappush(minPath, (cur_cost + cost, dst))

        # failure case: not every vertex was seen
        return -1

    def networkDelayTimeBellmanFord(
        self, times: List[List[int]], n: int, k: int
    ) -> int:
        """
        Bellman Ford Algorithm

        Basically, given a ceiling on iterations, traverse every single edge every single time while updating the state at each go.
        So the optimal solution at the nth iteration will have used the state from the n-1th iteration, etc.
        """
        dist = [float("inf")] * n
        dist[k - 1] = 0
        for _ in range(n - 1):
            for u, v, w in times:
                if dist[u - 1] + w < dist[v - 1]:
                    dist[v - 1] = dist[u - 1] + w
        max_dist = max(dist)
        return max_dist if max_dist < float("inf") else -1  # type: ignore

    def networkDelayTimeShortestPathFasterAlgorithm(
        self, times, n, k
    ):  # 67.4% time, 16.6% memory
        """
        Shortest Path Algo that only traverses an edge if it improves on existing path
        """
        adj = defaultdict(list)
        for u, v, w in times:
            adj[u].append((v, w))

        dist = {node: float("inf") for node in range(1, n + 1)}
        q = deque([(k, 0)])
        dist[k] = 0

        while q:
            node, time = q.popleft()
            if dist[node] < time:
                continue
            for nei, w in adj[node]:
                if time + w < dist[nei]:
                    dist[nei] = time + w
                    q.append((nei, time + w))

        res = max(dist.values())
        return res if res < float("inf") else -1


networkDelayTime = Solution()
networkDelayTime.networkDelayTime([[2, 1, 1], [2, 3, 1], [3, 4, 1]], 4, 2)

2

**778. Swim in Rising Water**


In [11]:
class Solution:
    def swimInWater(self, grid: List[List[int]]) -> int:  # 55 % time, 18% memory
        """
        Modified Djikstra's according to NeetCode so Ima give it a shot
        The minHeap will have to sort on "max height of path"
        Then at every iteration traverse the path that currently has the smallest max height

        max height is analogous to path cost in this case. we can move infinite distance instantly so length of path is irrelevant.

        A good line of thinking is that you are running the algo on the vertices not on the weights

        NOTE: Same as in Word Ladder:
        Moving the visited.add(word) from the outer for loop (under the success condition) accelerates it a lot (27% to 55% time)
        Because there are a bunch of heap additions that shouldn't be happening.
        """
        ROWS = len(grid)
        COLS = len(grid[0])

        # store the max height of path and the index that we have currently reached, the index is needed because we can only move in the cardinal directions
        # and we don't want to revisit indices for no reason.
        minHeap = [(grid[0][0], 0, 0)]
        heapq.heapify(minHeap)
        visited = set([(0, 0)])

        neighbors = [(1, 0), (-1, 0), (0, 1), (0, -1)]

        while minHeap:
            # while loop will always return
            curMaxHeight, row, col = heapq.heappop(minHeap)
            if row == ROWS - 1 and col == COLS - 1:
                return curMaxHeight

            for drow, dcol in neighbors:
                nrow, ncol = row + drow, col + dcol
                if (
                    nrow not in [-1, ROWS]
                    and ncol not in [-1, COLS]
                    and (nrow, ncol) not in visited
                ):
                    heapq.heappush(
                        minHeap, (max(curMaxHeight, grid[nrow][ncol]), nrow, ncol)
                    )
                    visited.add((nrow, ncol))

        return 0  # this is just here for the type hint

    # 15% time, 5$ memory
    def swimInWaterBinarySearch(self, grid: List[List[int]]) -> int:
        n = len(grid)
        minHeight = maxHeight = grid[0][0]
        for i in range(n):
            for j in range(n):
                if grid[i][j] < minHeight:
                    minHeight = grid[i][j]
                elif grid[i][j] > maxHeight:
                    maxHeight = grid[i][j]

        seen = set()

        def dfsCanReachEnd(i: int, j: int, target: int):
            if i in [-1, n] or j in [-1, n] or (i, j) in seen or grid[i][j] > target:
                # breaks one of the following conditions:
                # position out of bounds
                # position already visited
                # height is larger than the current searches max allowable height
                return False

            elif i == n - 1 and j == n - 1:
                # if the final bottom right position doesn't break the target height constraint return True
                return True

            # check if you can reach the end from any of the 4 children
            seen.add((i, j))
            canReachEnd = (
                dfsCanReachEnd(i + 1, j, target)
                or dfsCanReachEnd(i - 1, j, target)
                or dfsCanReachEnd(i, j + 1, target)
                or dfsCanReachEnd(i, j - 1, target)
            )

            return canReachEnd

        left, right = minHeight, maxHeight
        while left < right:
            targetHeight = (left + right) // 2
            if dfsCanReachEnd(0, 0, targetHeight):
                # check if you can reach the end with a smaller max height
                right = targetHeight
            else:
                left = targetHeight + 1

            # CRUCIAL to clear the seen set
            seen.clear()

        return right


swimInWater = Solution()
print(swimInWater.swimInWaterBinarySearch([[0, 2], [1, 3]]))
print(
    swimInWater.swimInWaterBinarySearch(
        [
            [0, 1, 2, 3, 4],
            [24, 23, 22, 21, 5],
            [12, 13, 14, 15, 16],
            [11, 17, 18, 19, 20],
            [10, 9, 8, 7, 6],
        ]
    )
)

3
16


**787. Cheapest Flights Within K Stops**

Bellman-Ford shortest path algorithm


In [ ]:
class Solution:
    """
    Seems like another djikstra's question. It is but NeetCode's video solution uses Bellman-ford because it is faster (he has the djikstra written solution)

    BFS while keeping track of the cost to reach each node you process
    """

    # 70% time, 14% memory
    def findCheapestPrice(
        self, n: int, flights: List[List[int]], src: int, dst: int, k: int
    ) -> int:
        graph = defaultdict(list)
        costs = {i: 1 << 28 for i in range(n)}
        for a, b, cost in flights:
            graph[a].append((cost, b))

        # BFS
        stack = deque([(0, src)])  # (pathCost, cur)

        for numLayovers in range(-1, k + 1):
            # start off with -1 layovers since the path to travel only has the src in it
            for _ in range(len(stack)):
                pathCost, cur = stack.popleft()
                # if another path to the same node was found to be cheaper, stop traversing this branch
                if pathCost > costs[cur]:
                    continue
                costs[cur] = pathCost

                if numLayovers == k:
                    # no need to do the append operations
                    continue

                for cost, nxt in graph[cur]:
                    stack.append((pathCost + cost, nxt))

        if costs[dst] == 1 << 28:
            return -1
        return costs[dst]


findCheapestPrice = Solution()
print(
    findCheapestPrice.findCheapestPrice(
        4,
        [[0, 1, 100], [1, 2, 100], [2, 0, 100], [1, 3, 600], [2, 3, 200]],
        0,
        3,
        1,
    )
)

print(
    findCheapestPrice.findCheapestPrice(
        13,
        [
            [11, 12, 74],
            [1, 8, 91],
            [4, 6, 13],
            [7, 6, 39],
            [5, 12, 8],
            [0, 12, 54],
            [8, 4, 32],
            [0, 11, 4],
            [4, 0, 91],
            [11, 7, 64],
            [6, 3, 88],
            [8, 5, 80],
            [11, 10, 91],
            [10, 0, 60],
            [8, 7, 92],
            [12, 6, 78],
            [6, 2, 8],
            [4, 3, 54],
            [3, 11, 76],
            [3, 12, 23],
            [11, 6, 79],
            [6, 12, 36],
            [2, 11, 100],
            [2, 5, 49],
            [7, 0, 17],
            [5, 8, 95],
            [3, 9, 98],
            [8, 10, 61],
            [2, 12, 38],
            [5, 7, 58],
            [9, 4, 37],
            [8, 6, 79],
            [9, 0, 1],
            [2, 3, 12],
            [7, 10, 7],
            [12, 10, 52],
            [7, 2, 68],
            [12, 2, 100],
            [6, 9, 53],
            [7, 4, 90],
            [0, 5, 43],
            [11, 2, 52],
            [11, 8, 50],
            [12, 4, 38],
            [7, 9, 94],
            [2, 7, 38],
            [3, 7, 88],
            [9, 12, 20],
            [12, 0, 26],
            [10, 5, 38],
            [12, 8, 50],
            [0, 2, 77],
            [11, 0, 13],
            [9, 10, 76],
            [2, 6, 67],
            [5, 6, 34],
            [9, 7, 62],
            [5, 3, 67],
        ],
        10,
        1,
        10,
    )
)

700
-1


**Alien Dictionary**

Want to find the alphabet used (in order) based on a bunch of words given.

Basically, build a graph where each node point to all letters that it comes before, then topological sort.
If you cannot, return `""`

Clever graph build and topological sort.


In [ ]:
class Solution:
    def foreignDictionary(self, words: List[str]) -> str:
        """
        The list of words is sorted lexicographically, this means the first letter is the first letter of the alphabet
        building the graph is non-trivial though.

        The way you gain information from the order of words is by finding the first point of difference between two words and adding that relation to your graph
        """

        graph = {char: set() for word in words for char in word}

        # build graph from each word pair in succession:
        for left in range(len(words) - 1):
            w_left, w_right = words[left], words[left + 1]
            minLength = min(len(w_left), len(w_right))

            if len(w_left) > len(w_right) and w_left[:minLength] == w_right[:minLength]:
                # a prefix should always come first, exit a failure case early
                return ""

            for i in range(len(w_left)):
                if w_left[i] != w_right[i]:
                    graph[w_left[i]].add(w_right[i])
                    break

        visit = {}  # False: finished being visited (can ignore), True: being visited (cycle)
        res = []

        def dfsReturnsHasCycle(c: str):
            if c in visit:
                hasCycle = visit[
                    c
                ]  # True if in current path, False if already processed in earlier path traversal
                return hasCycle

            visit[c] = True
            for nxt in graph[c]:
                if dfsReturnsHasCycle(nxt):
                    return True  # hasCycle
            visit[c] = False
            res.append(c)
            return False

        # need to try adding every node to the result:
        for c in graph:
            if dfsReturnsHasCycle(c):
                return ""

        # add any characters that have no information on lexicographical order (the list comprehension)
        return "".join(reversed(res))


foreignDictionary = Solution()
foreignDictionary.foreignDictionary(["hrn", "hrf", "hrfa", "er", "enn", "rfnn"])

'ahernf'

**1368. Minimum Cost to Make at Least One Valid Path in a Grid**


In [ ]:
class Solution:
    # 75% time, 62% memory (40 ms faster by using a matrix of booleans instead of a seen hashset)
    def minCost(self, grid: List[List[int]]) -> int:
        """
        Some variation of djikstra's but with mutable weights.
        At every position you push all possible moves to the minHeap
        Whenever the minimum cost move is to a block already visited, you just trim the heap
        """

        ROWS, COLS = len(grid), len(grid[0])
        minHeap = [(0, 0, 0)]  # entry is (cost, i, j)
        visited = [[False for _ in range(COLS)] for _ in range(ROWS)]
        add = [1, 1, 1, 1]
        while minHeap:
            cost, i, j = heapq.heappop(minHeap)
            if i == ROWS - 1 and j == COLS - 1:
                return cost
            visited[i][j] = True

            # mutate the add array for cost
            add[grid[i][j] - 1] -= 1

            # now lay out the options
            if j + 1 < COLS and not visited[i][j + 1]:
                heapq.heappush(minHeap, (cost + add[0], i, j + 1))
            if j - 1 >= 0 and not visited[i][j - 1]:
                heapq.heappush(minHeap, (cost + add[1], i, j - 1))
            if i + 1 < ROWS and not visited[i + 1][j]:
                heapq.heappush(minHeap, (cost + add[2], i + 1, j))
            if i - 1 >= 0 and not visited[i - 1][j]:
                heapq.heappush(minHeap, (cost + add[3], i - 1, j))

            # undo the mutate
            add[grid[i][j] - 1] += 1

            # trim the heap
            while visited[minHeap[0][1]][minHeap[0][2]]:
                heapq.heappop(minHeap)

        return 0  # just for the type hint


minCost = Solution()
print(minCost.minCost([[1, 1, 1, 1], [2, 2, 2, 2], [1, 1, 1, 1], [2, 2, 2, 2]]))
print(minCost.minCost([[1, 1, 3], [3, 2, 2], [1, 1, 4]]))
print(minCost.minCost([[1, 2], [4, 3]]))

3
0
1


**407. Trapping Rain Water 2**

Need to use djikstra's for the minimum modification since 3d space boundary condition is a contour not just 2 pointers like in trapping rain water 1


In [32]:
class Solution:
    # 18% time, 32% memory (195 ms runtime)
    def trapRainWater(self, heightMap: List[List[int]]) -> int:
        """
        The intuition is similar to Trapping Rain Water 1

        You start at the boundaries, but instead of pointers, you push them to a minheap using their height as the weight.
        This way you get the most limiting boundary node when you pop from the heap
        When processing a node:

        First node being processed will be a 1.
        - I think you have to push all of its neighbours to the heap
        - Update the global limiting factor based on the smallest value in the heap (ZERO)
            - This needs special treatment for boundary nodes
        - Add the amount of rain water trapped on top of the 1 (which is ZERO in this case, since it's boundary and it has neighbour of ZERO)

        Next node processed is 0.
        - The global limiting factor is still a 1, since the smallest value in the heap was a zero when the check was made
        - So we can trap 1 water
        ```
        2 1 2
        2 0 1
        1 1 1
        ```

        NOTE: not using a visited matrix, just gonna overwrite the heightMap
        """
        ROWS, COLS = len(heightMap), len(heightMap[0])
        if ROWS <= 2 or COLS <= 2:
            # need a middle row or column to trap anything lol
            return 0

        minHeap = []  # (height, i, j)
        for i in range(ROWS):
            # push the left and right borders
            heapq.heappush(minHeap, (heightMap[i][0], i, 0))
            heapq.heappush(minHeap, (heightMap[i][COLS - 1], i, COLS - 1))
        for j in range(COLS):
            # push the top and bottom borders
            heapq.heappush(minHeap, (heightMap[0][j], 0, j))
            heapq.heappush(minHeap, (heightMap[ROWS - 1][j], ROWS - 1, j))

        limit = 0  # boundary limit
        result = 0
        while minHeap:
            height, i, j = heapq.heappop(minHeap)
            heightMap[i][j] = -1  # mark as visited

            # update the limit based on the current height
            # NOTE: there may be a smaller height in the heap but it isn't a limiting factor because it is not a boundary node
            # since if it was a boundary node, it would have been processed before the current node
            limit = max(limit, height)
            # border nodes cannot trap water
            if not (i == 0 or i == ROWS - 1 or j == 0 or j == COLS - 1):
                result += limit - height

            # add neighbours to heap
            # top
            if i - 1 >= 0 and heightMap[i - 1][j] != -1:
                heapq.heappush(minHeap, (heightMap[i - 1][j], i - 1, j))
            # bot
            if i + 1 < ROWS and heightMap[i + 1][j] != -1:
                heapq.heappush(minHeap, (heightMap[i + 1][j], i + 1, j))
            # left
            if j - 1 >= 0 and heightMap[i][j - 1] != -1:
                heapq.heappush(minHeap, (heightMap[i][j - 1], i, j - 1))
            # right
            if j + 1 < COLS and heightMap[i][j + 1] != -1:
                heapq.heappush(minHeap, (heightMap[i][j + 1], i, j + 1))

            # trim the heap
            while minHeap and heightMap[minHeap[0][1]][minHeap[0][2]] == -1:
                heapq.heappop(minHeap)

        return result

    # 91% time, 70% memory (112 ms runtime)
    def trapRainWaterFaster(self, heightMap: List[List[int]]) -> int:
        """
        Can use the fact that when you process a node, its neighbours can also be processed since they are necessarily not boundary nodes
        This way you can reduce your heap operations by 4x since you can mark nodes as visited right away

        The node leading to a neighbour will have information on the maximum surrounding height since that's how the algorithm works.
        """
        ROWS, COLS = len(heightMap), len(heightMap[0])
        if ROWS <= 2 or COLS <= 2:
            # need a middle row or column to trap anything lol
            return 0

        minHeap = []  # (height, i, j)
        for i in range(ROWS):
            # push the left and right borders
            heapq.heappush(minHeap, (heightMap[i][0], i, 0))
            heapq.heappush(minHeap, (heightMap[i][COLS - 1], i, COLS - 1))
            heightMap[i][0] = -1
            heightMap[i][COLS - 1] = -1
        for j in range(COLS):
            # push the top and bottom borders
            heapq.heappush(minHeap, (heightMap[0][j], 0, j))
            heapq.heappush(minHeap, (heightMap[ROWS - 1][j], ROWS - 1, j))
            heightMap[0][j] = -1
            heightMap[ROWS - 1][j] = -1

        limit = 0  # boundary limit
        result = 0
        while minHeap:
            height, i, j = heapq.heappop(minHeap)
            limit = max(limit, height)

            # add neighbours to heap
            # top
            if i - 1 >= 0 and heightMap[i - 1][j] != -1:
                heapq.heappush(minHeap, (heightMap[i - 1][j], i - 1, j))
                result += max(0, limit - heightMap[i - 1][j])
                # mark visited after processing
                heightMap[i - 1][j] = -1
            # bot
            if i + 1 < ROWS and heightMap[i + 1][j] != -1:
                heapq.heappush(minHeap, (heightMap[i + 1][j], i + 1, j))
                result += max(0, limit - heightMap[i + 1][j])
                # mark visited after processing
                heightMap[i + 1][j] = -1
            # left
            if j - 1 >= 0 and heightMap[i][j - 1] != -1:
                heapq.heappush(minHeap, (heightMap[i][j - 1], i, j - 1))
                result += max(0, limit - heightMap[i][j - 1])
                # mark visited after processing
                heightMap[i][j - 1] = -1
            # right
            if j + 1 < COLS and heightMap[i][j + 1] != -1:
                heapq.heappush(minHeap, (heightMap[i][j + 1], i, j + 1))
                result += max(0, limit - heightMap[i][j + 1])
                # mark visited after processing
                heightMap[i][j + 1] = -1

        return result


trapRainWater = Solution()
print(
    trapRainWater.trapRainWaterFaster(
        [
            [1, 4, 3, 1, 3, 2],
            [3, 2, 1, 3, 2, 4],
            [2, 3, 3, 2, 3, 1],
        ]
    )
)
print(
    trapRainWater.trapRainWaterFaster(
        [
            [3, 3, 3, 3, 3],
            [3, 2, 2, 2, 3],
            [3, 2, 1, 2, 3],
            [3, 2, 2, 2, 3],
            [3, 3, 3, 3, 3],
        ]
    )
)
print(
    trapRainWater.trapRainWaterFaster(
        [
            [5, 5, 5, 1],
            [5, 1, 1, 5],
            [5, 1, 5, 5],
            [5, 2, 5, 8],
        ],
    )
)

4
10
3


**1976. Number of Ways to Arrive at Destination**


In [4]:
class Solution:
    def countPathsNeetCode(self, n: int, roads: List[List[int]]) -> int:
        """Trying to replicate neetcode's solution after watching the video"""
        return 0

    # 5% time, 5% memory (without early exiting getDistFrom0 - see docstring)
    # 25% time, 37% memory (with early exiting, cut runtime in half)
    def countPaths(self, n: int, roads: List[List[int]]) -> int:
        """
        First use any shortest path algorithm to get edges where dist[u] + weight = dist[v], here dist[x] is the shortest distance between node 0 and x

        Using those edges only the graph turns into a dag now we just need to know the number of ways to get from node 0 to node n - 1 on a dag using dp
        """

        def getDistFrom0(graph: List[List[Tuple[int, int]]]) -> List[int]:
            """
            One thing to note, since our heap is sorting by cost AND node value
            the moment we reach node n-1, we are done our search, all other paths are too long

            This is the Prim's algo for MST (Roy brought it up, i didn't realize)
            """
            minHeap = [(0, 0)]  # (cost, node) reverse of graph format
            visited = [False] * n
            result = [0] * n
            while minHeap:
                cost, u = heapq.heappop(minHeap)
                visited[u] = True
                result[u] = cost
                if u == n - 1:  # see docstring
                    return result
                # clean heap of visited nodes
                while minHeap and visited[minHeap[0][1]]:
                    heapq.heappop(minHeap)
                for v, t in graph[u]:
                    if not visited[v]:
                        heapq.heappush(minHeap, (cost + t, v))
            return result  # for type checker

        # undirected graph: node -> (neighbour, weight)
        graph = [[] for _ in range(n)]
        for u, v, t in roads:
            graph[u].append((v, t))
            graph[v].append((u, t))

        # get the shortest distance from 0 for all nodes
        distFrom0 = getDistFrom0(graph)

        # now get all edges that conform to the shortest distances from node 0
        # dist[u] + weight = dist[v]
        # this creates a DAG - directed acylic graph which we can DP on
        # since every edge in our DAG needs to be crossed to answer the question
        # we just count the number of times we reach a vertex
        # I think this is gonna be easiest with Kahn's algo since our graph is converging on n-1
        dag = [[] for _ in range(n)]
        inDegree = [0] * n
        for u, v, t in roads:
            if distFrom0[u] + t == distFrom0[v]:
                dag[u].append(v)
                inDegree[v] += 1
            elif distFrom0[v] + t == distFrom0[u]:
                dag[v].append(u)
                inDegree[u] += 1

        stack = deque()
        for u in range(n):
            if not inDegree[u]:
                stack.append(u)
        dp = [0] * n
        dp[0] = 1  # starting node
        MOD = pow(10, 9) + 7
        while stack:
            for _ in range(len(stack)):
                u = stack.popleft()
                for v in dag[u]:
                    inDegree[v] -= 1
                    dp[v] += dp[u]
                    dp[v] %= MOD
                    if inDegree[v] == 0:
                        stack.append(v)
        return dp[n - 1]


countPaths = Solution()
print(
    countPaths.countPaths(
        7,
        [
            [0, 6, 7],
            [0, 1, 2],
            [1, 2, 3],
            [1, 3, 3],
            [6, 3, 3],
            [3, 5, 1],
            [6, 5, 1],
            [2, 5, 1],
            [0, 4, 5],
            [4, 6, 2],
        ],
    )
)
print(countPaths.countPaths(2, [[1, 0, 10]]))

4
1
